In [ ]:
from abc import ABC, abstractmethod
import numpy as np

class Model(ABC):
    @abstractmethod
    def train(self, train_inputs, train_labels):
        pass
    
    @abstractmethod
    def predict(self, test_inputs):
        pass
        
class Gaussian(Model):
    def __init__(self):
    
    def train(self, train_inputs, train_labels):
        self.mean = np.mean(train_inputs)
        diff = train_inputs - self.mean
        outer_prod = np.outer(diff, diff)
    
    def predict(self, test_inputs):
    